In [14]:
#Imports
from keras.datasets import imdb

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics,activations

import matplotlib.pyplot as plt

In [2]:
#Downloading data from https://s3.amazonaws.com/text-datasets/imdb.npz

(xtrain,ytrain), (xtest, ytest) = imdb.load_data(num_words=10000)

In [3]:
#Exploring the dataset

print('xtrain shape', xtrain.shape)
print('ytrain shape', ytrain.shape)
print()
print('xtest shape', xtest.shape)
print('ytest shape', ytest.shape)
print()
print('xtrain first review as dictionary index', xtrain[1])
print()
print()
print('ytrain label', ytrain[0])

xtrain shape (25000,)
ytrain shape (25000,)

xtest shape (25000,)
ytest shape (25000,)

xtrain first review as dictionary index [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


ytrain label 1


In [4]:
#index to words mapping
word_index = imdb.get_word_index()

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [6]:
decode_review = ' '.join([reverse_word_index.get(i-3, '?') for i in xtrain[0]])
decode_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [7]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. 
    return results

x_train = vectorize_sequences(xtrain)
x_test = vectorize_sequences(xtest)

In [8]:
ytrain = np.asarray(ytrain).astype('float32')
ytest = np.asarray(ytest).astype('float32')

In [118]:
#model
model = models.Sequential()
model.add(layers.Dense(16, activation=activations.relu, input_shape=(10000,)))
model.add(layers.Dense(16, activation=activations.relu))
model.add(layers.Dense(1, activation=activations.sigmoid))

In [122]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss=losses.mse, metrics=['acc'])

In [123]:
x_val = x_train[:10000]
y_val = ytrain[:10000]

x_train_partial = x_train[10000:]
y_train_partial = ytrain[10000:]

In [124]:
history = model.fit(x_train_partial, y_train_partial, epochs=4, batch_size=512, validation_data=(x_val,y_val))
history_dict = history.history
history_dict.keys()
print(history.history['acc'][-1])
print(history.history['val_acc'][-1])

Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 2s 166us/step - loss: 0.1826 - acc: 0.8513 - val_loss: 0.1805 - val_acc: 0.8378
Epoch 2/4
15000/15000 [==============================] - 2s 131us/step - loss: 0.1679 - acc: 0.8691 - val_loss: 0.1698 - val_acc: 0.8484
Epoch 3/4
15000/15000 [==============================] - 2s 132us/step - loss: 0.1558 - acc: 0.8780 - val_loss: 0.1590 - val_acc: 0.8561
Epoch 4/4
15000/15000 [==============================] - 2s 132us/step - loss: 0.1449 - acc: 0.8831 - val_loss: 0.1500 - val_acc: 0.8615
0.8830666665077209
0.8614999994277954


dict_keys(['val_acc', 'loss', 'acc', 'val_loss'])

In [125]:
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(0, len(loss)+1)
epochs

range(0, 5)

In [126]:
%matplotlib
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'ro', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

Using matplotlib backend: TkAgg


In [127]:
plt.clf()      # clear figure# clear  
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()